# Nova Code Interpreter with LangChain

This notebook demonstrates how to use Amazon Nova 2.0's code interpreter system tool with LangChain's ChatBedrockConverse.

System tools are different from standard tools - the model can utilize these tools throughout its reasoning process and actually invoke the tool itself. This is in contrast with standard tool workflows where the model returns a tool call and the developer is responsible for invocation.

The Nova Code Interpreter system tool enables the Nova 2.0 model to execute Python code in a sandboxed environment for complex calculations and data processing.

## Setup

Install the required packages:

In [ ]:
%pip install -qU langchain-aws

## IAM Permissions

**Important:** To invoke system tools, your IAM role must have the `bedrock:InvokeTool` permission in addition to `bedrock:InvokeModel`.

## Initialize Model

Note: The read timeout is especially important for code interpreter since code execution can take time.

In [ ]:
from botocore.config import Config
from langchain_aws import ChatBedrockConverse
from langchain_aws.chat_models import NovaCodeInterpreterTool

# Configure extended timeouts for code execution
config = Config(
    connect_timeout=3600,  # 60 minutes
    read_timeout=3600,     # 60 minutes
    retries={'max_attempts': 1}
)

# Initialize model
model = ChatBedrockConverse(
    model="amazon.nova-2-lite-v1:0",
    region_name="us-east-1",
    temperature=1.0,
    max_tokens=10000,
    config=config,
    additional_model_request_fields={
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "low"
        }
    }
)

# Bind the code interpreter tool
model_with_code = model.bind_tools([NovaCodeInterpreterTool()])

## Basic Code Execution Example

In [ ]:
# Simple calculation
response = model_with_code.invoke("Calculate 7^6 using the code interpreter")
print(response.content)

## Alternative: Using String Name

You can also use the string name directly:

In [ ]:
# Using string name instead of tool class
model_with_code = model.bind_tools(["nova_code_interpreter"])

response = model_with_code.invoke("What is the square root of 475878756857?")
print(response.content)

## Understanding the Response

Let's examine the different parts of the response to see the reasoning, code execution, and results:

In [ ]:
# Execute a calculation and examine the response structure
response = model_with_code.invoke("Calculate the factorial of 20")

for i, block in enumerate(response.content):
    if isinstance(block, dict):
        print(f"\n=== Block {i} ===")
        if 'type' in block:
            print(f"Type: {block['type']}")
            
            if block['type'] == 'reasoning_content':
                print("Reasoning:", block['reasoning_content']['text'][:200], "...")
            elif block['type'] == 'tool_use':
                print("Tool:", block.get('name'))
                if 'input' in block:
                    print("Code:", block['input'])
            elif block['type'] == 'tool_result':
                print("Tool Result:")
                if 'content' in block:
                    for content in block['content']:
                        if isinstance(content, dict) and 'json' in content:
                            result = content['json']
                            print(f"  stdout: {result.get('stdOut', '')}")
                            print(f"  stderr: {result.get('stdErr', '')}")
                            print(f"  exitCode: {result.get('exitCode', 0)}")
                            print(f"  isError: {result.get('isError', False)}")
            elif block['type'] == 'text':
                print("Final Answer:", block['text'])

## Tool Result Format

The code interpreter returns results in this format:
```json
{
    "stdOut": "output from print statements",
    "stdErr": "error messages if any",
    "exitCode": 0,
    "isError": false
}
```

If errors occur, `stdErr` will contain the error message, and `exitCode` and `isError` will reflect the error state.

## Streaming with Code Interpreter

You can stream responses to see the reasoning, code execution, and results in real-time:

In [ ]:
print("Streaming response:\n")

current_type = None
for chunk in model_with_code.stream("Calculate the first 10 Fibonacci numbers"):
    if chunk.content:
        for block in chunk.content:
            if isinstance(block, dict) and 'type' in block:
                # Print header when type changes
                if block['type'] != current_type:
                    current_type = block['type']
                    print(f"\n\n=== {current_type.upper()} ===")
                
                # Stream the content
                if block['type'] == 'reasoning_content':
                    print(block['reasoning_content'].get('text', ''), end='', flush=True)
                elif block['type'] == 'text':
                    print(block.get('text', ''), end='', flush=True)
                elif block['type'] == 'tool_use':
                    if 'name' in block:
                        print(f"Tool: {block['name']}")
                    if 'input' in block:
                        print(f"Code: {block['input']}")
                elif block['type'] == 'tool_result':
                    if 'content' in block:
                        print(block['content'])

## Complex Calculations and Data Processing

The code interpreter can handle more complex tasks:

In [ ]:
# Statistical analysis
response = model_with_code.invoke(
    "Calculate the mean, median, and standard deviation of these numbers: "
    "[23, 45, 67, 12, 89, 34, 56, 78, 90, 11]"
)
print(response.content)

In [ ]:
# Data transformation
response = model_with_code.invoke(
    "Create a list of prime numbers between 1 and 100, "
    "then calculate their sum and average"
)
print(response.content)

## Adjusting Reasoning Effort

For more complex problems, you can increase the reasoning effort:

In [ ]:
# Model with medium reasoning effort
model_medium = ChatBedrockConverse(
    model="amazon.nova-2-lite-v1:0",
    max_tokens=10000,
    config=config,
    additional_model_request_fields={
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "medium"
        }
    }
)

model_with_code_medium = model_medium.bind_tools([NovaCodeInterpreterTool()])

# Complex problem requiring more reasoning
response = model_with_code_medium.invoke(
    "Write code to solve the Tower of Hanoi problem for 5 disks "
    "and show the number of moves required"
)
print(response.content)

## Combining Code Interpreter with Structured Output

You can combine code execution with structured output for clean results:

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class StatisticalAnalysis(BaseModel):
    """Statistical analysis results"""
    mean: float = Field(description="The mean value")
    median: float = Field(description="The median value")
    std_dev: float = Field(description="The standard deviation")
    min_value: float = Field(description="The minimum value")
    max_value: float = Field(description="The maximum value")

# Create a model with both code interpreter and structured output
structured_model = model_with_code.with_structured_output(StatisticalAnalysis)

result = structured_model.invoke(
    "Analyze these numbers: [15, 23, 45, 67, 12, 89, 34, 56, 78, 90, 11, 44]"
)

print(f"Mean: {result.mean}")
print(f"Median: {result.median}")
print(f"Standard Deviation: {result.std_dev}")
print(f"Min: {result.min_value}")
print(f"Max: {result.max_value}")

## Error Handling

The model can detect and handle code execution errors:

In [ ]:
# This will cause the model to write code that might have issues
# and potentially self-correct
response = model_with_code.invoke(
    "Calculate 1 divided by 0 and handle any errors appropriately"
)
print(response.content)